## Uporaba Orange za klasifikacijo

## Using Orange for classification

Orange nudi funkcije za izdelavo klasifikacijskih modelov, njihovo ocenjevanje in točkovanje.

Orange implements function for construction of classification models, their evaluation and scoring. 

In [ ]:
import Orange
import numpy as np
from numpy import random

### Učni algoritmi (`learners`) in klasifikatorji (`classifiers`)

### Learners and Classifiers

Klasifikacija uporablja dve vrsti objektov: učne algoritme in klasifikatorje. Učni lagoritmi
upoštevajo podatke, označene z razredom, in vrnejo klasifikator. Glede na prve tri podatkovne primere, klasifikatorji vrnejo indekse predvidenega razreda:

Classification uses two types of objects: learners and classifiers. Learners consider class-labeled data and return a classifier. Given the first three data instances, classifiers return the indexes of predicted class:

In [ ]:
data = Orange.data.Table("voting")
learner = Orange.classification.LogisticRegressionLearner()
classifier = learner(data)
classifier(data[:3])

Zgoraj smo prebrali podatke, konstruirali logistično regresijo in mu dali podatkovni niz za izdelavo klasifikatorja ter ga uporabili za predvidevanje razreda prvih treh podatkovnih primerkov. Te koncepte uporabljamo tudi v naslednji kodi, ki napoveduje razrede izbranih treh primerkov v naboru podatkov:

Above, we read the data, constructed a logistic regression learner, gave it the dataset to construct a classifier, and used it to predict the class of the first three data instances. We also use these concepts in the following code that predicts the classes of the selected three instances in the dataset:

In [ ]:
learner = Orange.classification.LogisticRegressionLearner()
classifier = learner(data)
c_values = data.domain.class_var.values
for d in data[5:8]:
    c = classifier(d)
    print("{}, originally {}".format(c_values[int(classifier(d)[0])],
                                     d.get_class()))

Logistična regresija je v drugem primeru naredila napako, sicer pa je napovedana pravilno. Ni čudno, saj so bili to tudi podatki, na katerih se je učila. Naslednja koda upošteva število takih napak v celotnem naboru podatkov:

Logistic regression has made a mistake in the second case, but otherwise predicted correctly. No wonder, since this was also the data it trained from. The following code counts the number of such mistakes in the entire dataset:

In [ ]:
data = Orange.data.Table("voting")
learner = Orange.classification.LogisticRegressionLearner()
classifier = learner(data)
print(np.sum(data.Y != classifier(data)))

To število nam ne pove dosti o natančnosti klasifikatorja. Prejšnji teden smo spoznali boljši način. 

This number does not tell us much about the accuracy of the classifier. Last week we learnt of a better way.

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(data.Y, classifier(data)))

Če želite izvedeti, kakšna je verjetnost, da razvrščevalec dodeli, recimo, razred za demokrate, moramo klasifikator poklicati z dodatnim parametrom, ki podaja vrsto klasifikacije.

To find out what is the probability that the classifier assigns to, say, democrat class, we need to call the classifier with additional parameter that specifies the classification output type.

In [ ]:
data = Orange.data.Table("voting")
learner = Orange.classification.LogisticRegressionLearner()
classifier = learner(data)
target_class = 1
print("Probabilities for %s:" % data.domain.class_var.values[target_class])
probabilities = classifier(data, 1)
for p, d in zip(probabilities[5:8], data[5:8]):
    print(p[target_class], d.get_class())

Orange vključuje različne klasifikacijske algoritme, večina jih je ovitih iz `scikit-learn`, med drugim:

* logistična regresija (`Orange.classification.LogisticRegressionLearner`)
* k najbližjih sosedov (`Orange.classification.knn.KNNLearner`)
* support vector machines (npr. `Orange.classification.svm.LinearSVMLearner`)
* klasifikacijska drevesa (`Orange.classification.tree.SklTreeLearner`)
* naključni gozd (`Orange.classification.RandomForestLearner`)

Nekatere od teh so vključene v kodo, ki ocenjuje verjetnost ciljnega razreda na preskusnih podatkih. Tokrat se podatki za učenje in testiranje razlikujejo:

Orange includes a variety of classification algorithms, most of them wrapped from `scikit-learn`, including:

* logistic regression (`Orange.classification.LogisticRegressionLearner`)
* k-nearest neighbors (`Orange.classification.knn.KNNLearner`)
* support vector machines (say, `Orange.classification.svm.LinearSVMLearner`)
* classification trees (`Orange.classification.tree.SklTreeLearner`)
* random forest (`Orange.classification.RandomForestLearner`)

Some of these are included in the code that estimates the probability of a target class on a testing data. This time, training and test datasets are disjoint:

In [ ]:
import random
random.seed(42)
data = Orange.data.Table("voting")
test = Orange.data.Table(data.domain, random.sample(data, 5))
train = Orange.data.Table(data.domain, [d for d in data if d not in test])

tree = Orange.classification.tree.TreeLearner(max_depth=3)
knn = Orange.classification.knn.KNNLearner(n_neighbors=3)
lr = Orange.classification.LogisticRegressionLearner(C=0.1)

learners = [tree, knn, lr]
classifiers = [learner(train) for learner in learners]

target = 0
print("Probabilities for %s:" % data.domain.class_var.values[target])
print("original class ", " ".join("%-5s" % l.name for l in classifiers))

c_values = data.domain.class_var.values
for d in test:
    print(("{:<15}" + " {:.3f}"*len(classifiers)).format(
        c_values[int(d.get_class())],
        *(c(d, 1)[0][target] for c in classifiers)))

##### Vprašanje 5-3-1
Preizkusi različne klasifikatorje. Kateri deluje najbolje na danih podatkih?

##### Question 5-3-1
Test different classifiers. Which works best on given data?

### Krivulja ROC in površina pod krivuljo (Area Under Curve - AUC)

### ROC curve and Area Under Curve (AUC)

Krivulja ROC je način, kako oceniti kvaliteto klasifikacije. Tipično na y os označimo delež resnično pozitivnih (TP rate) in na x os delež lažno pozitivnih (FP rate). Zgornji levi kot nam tako predstavlja idealni model brez lažno pozitivnih primerov in s 100% deležem resnično pozitivnih.

The ROC curve is a way to assess the quality of the classification. Typically, the y axis indicates true positive rate, and the x axis the false positive rate. The upper left corner represents an ideal model without false positives and with a 100% true positive rate.

In [ ]:
random.seed(42)  # zagotovi ponovljivost naključnih rezultatov

data = Orange.data.Table('titanic')
inxs = list(range(len(data)))
n = len(inxs)

random.shuffle(inxs)

data_training = data[inxs[:n//2]]
data_test     = data[inxs[n//2:]]

learner = Orange.classification.LogisticRegressionLearner()
classifier = learner(data_training)
classifier(data_test)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'jpg'
import matplotlib.pyplot as plt
plt.style.use('PR.mplstyle')

from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(classifier(data_test), data_test.Y)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

##### Vprašanje 5-3-2
Poišči modele z večjo površino pod krivuljo.

##### Question 5-3-2
Find models with a larger area under curve.

### Prečno preverjanje

### Cross-Validation

Ocenjevanje točnosti klasifikatorjev na učni množici, kot smo to storili zgoraj, služi samo za predstavitvene namene. Vsako mero natančnosti, je treba oceniti na neodvisni množici. Tak je tudi postopek, imenovan prečno preverjanje, ki povpreči vrednotenje po več izvajanjih, vsakič z drugačno učno in testno množico, ki so vzorčeni iz prvotnega nabora podatkov:

Validating the accuracy of classifiers on the training data, as we did above, serves demonstration purposes only. Any performance measure that assesses accuracy should be estimated on the independent test set. Such is also a procedure called cross-validation, which averages the evaluation scores across several runs, each time considering a different training and test subsets as sampled from the original dataset:

In [ ]:
data = Orange.data.Table("titanic")
lr = Orange.classification.LogisticRegressionLearner()
res = Orange.evaluation.CrossValidation(data, [lr], k=5)
print("Accuracy: %.3f" % Orange.evaluation.scoring.CA(res)[0])
print("AUC:      %.3f" % Orange.evaluation.scoring.AUC(res)[0])

Prečno preverjanje pričakuje seznam učencev. Ocenjevalci uspešnosti vrnejo tudi seznam točk, po enega za vsakega učenca. V zgornji skripti je bil samo en učenec (lr), zato je bil vrnjen seznam dolžine ena. Skripta ocenjuje natančnost klasifikacije in površino pod krivuljo ROC.

Cross-validation is expecting a list of learners. The performance estimators also return a list of scores, one for every learner. There was just one learner (lr) in the script above, hence the array of length one was returned. The script estimates classification accuracy and area under ROC curve.

Naslednja koda prečno preverja tri učence na zbirki podatkov titanic.

The following code cross-validates three learners on the titanic dataset.

In [ ]:
data = Orange.data.Table("titanic")
tree = Orange.classification.tree.TreeLearner(max_depth=3)
knn = Orange.classification.knn.KNNLearner(n_neighbors=3)
lr = Orange.classification.LogisticRegressionLearner(C=0.1)
learners = [tree, knn, lr]

print(" "*9 + " ".join("%-4s" % learner.name for learner in learners))
res = Orange.evaluation.CrossValidation(data, learners, k=5)
print("Accuracy %s" % " ".join("%.2f" % s for s in Orange.evaluation.CA(res)))
print("AUC      %s" % " ".join("%.2f" % s for s in Orange.evaluation.AUC(res)))

Seveda lahko tudi prečno preverimo učence istega tipa z različnimi parametri.

Of course, we can also cross-check learners of the same type with different parameters.

In [ ]:
data = Orange.data.Table("titanic")
knn3 = Orange.classification.knn.KNNLearner(n_neighbors=3)
knn5 = Orange.classification.knn.KNNLearner(n_neighbors=5)
knn7 = Orange.classification.knn.KNNLearner(n_neighbors=7)
learners = [knn3, knn5, knn7]

print(" "*10 + " ".join("%-4s" % learner.name for learner in learners))
res = Orange.evaluation.CrossValidation(data, learners, k=5)
print("neighbors " + " ".join(" %i  " % i for i in [3, 6, 9]))
print("Accuracy  %s" % " ".join("%.2f" % s for s in Orange.evaluation.CA(res)))
print("AUC       %s" % " ".join("%.2f" % s for s in Orange.evaluation.AUC(res)))

##### Vprašanje 5-3-3
Izberi metodo klasificiranja in s prečnim preverjanjem testiraj različne vrednosti parametrov.

##### Question 5-3-3
Select a classifying method and use cross validation to evaluate different parameter values.